# NMEG SPEI

Calculate SPEI (Standardized precipitation-evaporation index) for NMEG data and historical SPEI at the sites.

## First calculate SPEI for NMEG data period

In [1]:
# Load daily data files
source('../r_functions/load_nmeg.r')

seg <- daily_to_xts(get_daily_file('Seg', 'aflx', make_new=FALSE))
ses <- daily_to_xts(get_daily_file('Ses', 'aflx', make_new=FALSE))
sen <- daily_to_xts(get_daily_file('Sen', 'aflx', make_new=FALSE))
# wjs has some empty rows that are a problem
wjs <- daily_to_xts(get_daily_file('Wjs', 'aflx', make_new=FALSE))
nonemptyrows <- rowSums(is.na(wjs)) != ncol(wjs)
firstnonempty <- min(which(nonemptyrows))
wjs <- wjs[firstnonempty:nrow(wjs),]

mpj <- daily_to_xts(get_daily_file('Mpj', 'aflx', make_new=FALSE))
mpg <- daily_to_xts(get_daily_file('Mpg', 'aflx', make_new=FALSE))
vcp <- daily_to_xts(get_daily_file('Vcp', 'aflx', make_new=FALSE))
vcm <- daily_to_xts(get_daily_file('Vcm', 'aflx', make_new=FALSE))

head(wjs)

Loading required package: zoo

Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric



           GPP_g_int RECO_g_int FC_F_g_int FC_F_g_nightint FC_F_g_dayint
2007-01-01        NA         NA         NA              NA            NA
2007-01-02        NA         NA         NA              NA            NA
2007-01-03        NA         NA         NA              NA            NA
2007-01-04        NA         NA         NA              NA            NA
2007-01-05        NA         NA         NA              NA            NA
2007-01-06        NA         NA         NA              NA            NA
           ET_mm_24hint_0 P_F_sum SUN_FLAG SUN_HR TA_F_avg RH_F_avg SW_IN_F_avg
2007-01-01             NA      NA        0      0       NA       NA          NA
2007-01-02             NA      NA        0      0       NA       NA          NA
2007-01-03             NA      NA        0      0       NA       NA          NA
2007-01-04             NA      NA        0      0       NA       NA          NA
2007-01-05             NA      NA        0      0       NA       NA          NA
2007-01-0

## Examine climatic water deficit for all sites

In [2]:
# Calculate climatic water deficit
sitelist <- list( seg, ses, sen, wjs, mpj, mpg, vcp, vcm )
sitenames <- list( 'Seg', 'Ses', 'Sen', 'Wjs', 'Mpj', 'Mpg', 'Vcp', 'Vcm' )
cwdiff <- data.frame()
for (i in 1:8){
    cwdiff <- cbind( cwdiff, apply.monthly(sitelist[[i]]$P_F, FUN=sum) - 
                    apply.monthly(sitelist[[i]]$PET_mm_dayint, FUN=sum))
}
colnames(cwdiff) <- sitenames
head(cwdiff)
#plot.xts(cwdiff, screens=1)


                 Seg        Ses Sen       Wjs Mpj Mpg         Vcp         Vcm
2007-01-31  -4.63156  -3.699381  NA        NA  NA  NA   12.945333   63.433955
2007-02-28 -14.08761 -11.666628  NA        NA  NA  NA    6.175264   24.929334
2007-03-31 -11.93482 -10.793086  NA        NA  NA  NA   25.692094   14.609039
2007-04-30 -34.14474 -23.094100  NA        NA  NA  NA  -29.317850    3.706719
2007-05-31 -64.73956 -36.281533  NA        NA  NA  NA  -31.119326    2.300833
2007-06-30 -59.86259 -65.350091  NA -33.88382  NA  NA -129.664641 -101.268706

## Load packages for calculating SPEI
This package is on CRAN [here](https://cran.r-project.org/web/packages/SPEI/index.html). Website for the project is <http://sac.csic.es/spei/index.html>

If not installed use "install.packages('SPEI')"

In [9]:
install.packages('SPEI' , repos='http://cran.us.r-project.org')

package 'SPEI' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\alex\AppData\Local\Temp\RtmpKkktM2\downloaded_packages


In [10]:
library('SPEI')
library('xts')

Warning message:
"package 'SPEI' was built under R version 3.3.3"Loading required package: lmomco
Warning message:
"package 'lmomco' was built under R version 3.3.3"Loading required package: parallel
Loading required package: ggplot2

Attaching package: 'ggplot2'

The following object is masked _by_ '.GlobalEnv':

    mpg

# Package SPEI (1.7) loaded [try SPEINews()].


In [11]:
# Function for retrieving SPEI values from a dataframe at
# a number of different timesteps
get_spei_steps <- function(df, sitename, tstep='monthly',
                           int_steps=seq(1, 12), plot=TRUE){
    if (tstep=='monthly'){
        freq <- 12
        } else if (tstep=='weekly'){
        freq <- 52
    }
    # Get start date
    startmon <- as.yearmon(index(df[1]))
    startyr <- floor(as.numeric(startmon))
    startmon <- as.numeric( format( startmon, '%m'))
    # Calculate climatic water difference on a weekly or monthly scale
    if (tstep=='monthly'){
        cwdiff <- apply.monthly(df$P_F_sum, FUN=sum) - apply.monthly(df$PET_mm_dayint, FUN=sum)
    } else if (tstep=='weekly'){
        cwdiff <- apply.weekly(df$P_F_sum, FUN=sum) - apply.weekly(df$PET_mm_dayint, FUN=sum)
    }
    colnames(cwdiff) <- sitename
    for (i in 1:length(int_steps)){
        # Integration period for SPEI (in number of timesteps)
        SPEI_int_per <- int_steps[i]
        # Get spei for that integration period
        spei_int <- spei(ts(cwdiff, frequency=freq, start=c(startyr, startmon)), SPEI_int_per, na.rm=TRUE)
        if (plot){
            plot(spei_int)
        }
        # Extract the spei values from the returned object and make xts
        spei_int <- xts(as.vector(spei_int$fitted),  index(cwdiff))
        colnames(spei_int) <- paste('SPEI_', tstep, '_', as.character(SPEI_int_per), sep='')
        if (i==1){
            df_new <- spei_int
        } else if (i > 1) {
            df_new <- cbind(df_new, spei_int)
        }
    }
    print(head(df_new))
    
    # Somehow a funky entry at 2009-2-12 is created, remove
    #print(nrow(df_new))
    #df_new <- df_new[c('::2009-01-31', '2009-02-28::')]
    #print(nrow(df_new))
    
    # There may be both infinite and NA values in the output
    # Convert -Inf to NA
    print(sum(is.na(df_new)))
    df_new[!is.finite(df_new)] <- NA
    print(sum(is.na(df_new)))
    
    # Interpolate over NA values
    df_new_interp <- na.approx(df_new)
        
    return( list(df_new, df_new_interp) )
}

In [59]:
sitenum <- 3
int_steps1 <- seq(1, 12)
print( sitenames[[sitenum]] )
print( head(sitelist[[sitenum]]) )
#df <- sitelist[[sitenum]]
#cwdiff1 <- apply.weekly(df$P_F, FUN=sum) - apply.weekly(df$PET_mm_dayint, FUN=sum)
#spei_int1 <- spei(ts(cwdiff1, frequency=52, start=c(startyr, 1)), SPEI_int_per, na.rm=TRUE)
spei_weekly <- get_spei_steps(sitelist[[sitenum]], sitenames[[sitenum]], int_steps=int_steps1, tstep='monthly', plot=FALSE)
#print( spei_weekly[[2]] )

[1] "Sen"
           GPP_g_int RECO_g_int FC_F_g_int ET_mm_24hint_0 P_F_sum   TA_F_avg
2010-01-01 0.1136120  0.4259083 0.31229628      0.4032358       0 -2.5114543
2010-01-02 0.2161500  0.3143947 0.09824470      0.2736742       0 -0.2682929
2010-01-03 0.1204642  0.3600118 0.23954760      0.2097532       0 -0.9624908
2010-01-04 0.1580211  0.3750380 0.21701696      0.1573903       0 -3.0812138
2010-01-05 0.1367253  0.3438134 0.20708806      0.1526637       0 -3.3661277
2010-01-06 0.1897916  0.2673470 0.07755541      0.1364428       0 -0.6928112
           RH_F_avg SW_IN_F_avg RNET_F_avg VPD_F_avg  TA_F_min VPD_F_min
2010-01-01 59.81988    166.7249   45.91650 0.2537424 -10.21888  0.037451
2010-01-02 62.02385    146.0594   36.79920 0.2731999  -6.29779  0.035677
2010-01-03 61.37402    165.1616   37.36449 0.2668355  -7.10839  0.037133
2010-01-04 66.35104    164.8700   40.86090 0.2208267  -9.35950  0.027159
2010-01-05 67.14276    161.6243   40.18154 0.2360786 -10.76979  0.020663
2010-01-06 60

In [12]:
# Create monthly files

for (j in 1:length(sitelist)){
    df_j <- sitelist[[j]]
    sitenm <- sitenames[[j]]
    int_steps1 <- seq(1, 24)
    if(sitenm=='Sen'){
        int_steps1 <- seq(1, 23)# For some reason Sen has a problem with 24 month SPEI
    }
    spei_monthly <- get_spei_steps(df_j, sitenm, int_steps=int_steps1, tstep='monthly', plot=FALSE)
    outfile <- paste('../processed_data/spei/SPEI_monthly_US-',
                     sitenames[[j]], '.csv', sep='')
    write.zoo(spei_monthly[[1]], file = outfile,
              index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
    outfile <- paste('../processed_data/spei/SPEI_monthly_US-', sitenames[[j]],
                     '_nainterp.csv', sep='')
    write.zoo(spei_monthly[[2]], file = outfile,
              index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
}

           SPEI_monthly_1 SPEI_monthly_2 SPEI_monthly_3 SPEI_monthly_4
2007-01-31     0.04156617             NA             NA             NA
2007-02-28    -0.12070716     -0.1064771             NA             NA
2007-03-31     1.85041236      1.4715230      1.0124648             NA
2007-04-30     0.51953000      1.7169614      1.3470395     0.90620807
2007-05-31    -1.35111268     -0.6399993      0.2644214     0.15884830
2007-06-30    -0.70091164     -0.7699133     -0.4786267     0.05407007
           SPEI_monthly_5 SPEI_monthly_6 SPEI_monthly_7 SPEI_monthly_8
2007-01-31             NA             NA             NA             NA
2007-02-28             NA             NA             NA             NA
2007-03-31             NA             NA             NA             NA
2007-04-30             NA             NA             NA             NA
2007-05-31     0.21924463             NA             NA             NA
2007-06-30    -0.00976252     0.05557016             NA             NA
      

In [13]:
# Now weekly
#
# WARNING - note that weekly SPEI generates more NA values - not sure why
#
int_steps2 <- seq(4, 96, 4)
for (j in 1:length(sitelist)){
    df_j <- sitelist[[j]]
    spei_weekly <- get_spei_steps(df_j, 'Ses', int_steps=int_steps2, tstep='weekly', plot=FALSE)
    outfile <- paste('../processed_data/spei/SPEI_weekly_US-',
                     sitenames[[j]], '.csv', sep='')
    write.zoo(spei_weekly[[1]], file = outfile,
              index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
    outfile <- paste('../processed_data/spei/SPEI_weekly_US-', sitenames[[j]],
                     '_nainterp.csv', sep='')
    write.zoo(spei_weekly[[2]], file = outfile,
              index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
}

           SPEI_weekly_4 SPEI_weekly_8 SPEI_weekly_12 SPEI_weekly_16
2007-01-07            NA            NA             NA             NA
2007-01-14            NA            NA             NA             NA
2007-01-21            NA            NA             NA             NA
2007-01-28    -0.1120675            NA             NA             NA
2007-02-04    -0.4552684            NA             NA             NA
2007-02-11    -0.5131956            NA             NA             NA
           SPEI_weekly_20 SPEI_weekly_24 SPEI_weekly_28 SPEI_weekly_32
2007-01-07             NA             NA             NA             NA
2007-01-14             NA             NA             NA             NA
2007-01-21             NA             NA             NA             NA
2007-01-28             NA             NA             NA             NA
2007-02-04             NA             NA             NA             NA
2007-02-11             NA             NA             NA             NA
           SPEI_week

## Now calculate SPEI for historical periods (PRISM)

In [19]:
fpath <- 'C:/Research_Flux_Towers/Ancillary_met_data/PRISM_monthly/'
fname_ppt <- 'PRISM_Monthly_ppt_1981_2017.csv'
fname_tmean <-'PRISM_Monthly_tmean_1981_2017.csv'
#header <- read.csv(fname, skip=3, nrows=1)
df_ppt <- read.csv(paste(fpath, fname_ppt, sep=''))
df_tmean <- read.csv(paste(fpath, fname_tmean, sep=''))
df_ppt <- xts( df_ppt[,2:ncol(df_ppt)], as.Date(df_ppt$date))
df_tmean <- xts( df_tmean[,2:ncol(df_tmean)], as.Date(df_tmean$date))
head(df_ppt)
head(df_tmean)

           US.Wjs US.Mpj US.Mpg US.Seg US.Sen US.Ses US.Vcm US.Vcp US.Vcs
1981-01-31   3.88   5.38   5.26   5.38   6.10   5.58   4.21   4.39   5.40
1981-02-28   5.97   8.40   8.55   4.73   4.95   4.59  28.38  23.72  19.78
1981-03-31  16.98  24.01  24.16  16.08  16.50  16.11 161.74 136.64 111.88
1981-04-30  11.00  19.80  19.94  10.30  13.20  10.74  47.23  43.04  37.65
1981-05-31  10.12  16.72  15.86   9.75  10.13   9.26  88.62  76.18  58.08
1981-06-30  20.76  16.05  15.49  12.04  12.21  10.90  61.35  57.34  53.16

           US.Wjs US.Mpj US.Mpg US.Seg US.Sen US.Ses US.Vcm US.Vcp US.Vcs
1981-01-31  2.075  1.340  1.515  3.615  3.535  3.620 -2.020 -0.890 -0.705
1981-02-28  4.055  3.510  3.665  6.205  6.010  6.320 -2.580 -1.385 -1.360
1981-03-31  5.245  4.555  4.675  8.080  7.910  8.095 -2.415 -1.115 -0.825
1981-04-30 12.065 11.015 11.170 15.190 14.915 15.240  3.945  5.420  5.930
1981-05-31 14.975 13.525 13.695 18.585 18.305 18.615  6.125  7.640  8.135
1981-06-30 21.625 20.955 21.180 24.770 24.505 24.850 13.450 14.710 14.985

In [20]:
# Get site coordinates
coords <- read.csv('../site_coords.txt')
coords

sitecode,lat,lon
US-Wjs,34.42549,-105.8615
US-Mpj,34.43845,-106.2377
US-Mpg,34.44682,-106.2134
US-Seg,34.36233,-106.7019
US-Sen,34.35802,-106.6799
US-Ses,34.33494,-106.7442
US-Vcm,35.88845,-106.5321
US-Vcp,35.86423,-106.5967
US-Vcs,35.91927,-106.6142


## Calculate SPEI using thornwaite PET

In [21]:
plot_spei <- FALSE
int_steps1 <- seq(1, 24)

for (i in 1:length(sitenames)){
    # Get site name and latitude
    sitename1 <- paste('US-', sitenames[[i]], sep='')
    lat <- coords[coords$sitecode==sitename1, 2]
    sitename2 <- paste('US.', sitenames[[i]], sep='') # Format sitename for PRISM datasets
    # Calculate pet and climatic water diff with thornthwaite (convert to ts for this)
    ta <- ts(df_tmean[,sitename2], frequency=12)
    ppt <- ts(df_ppt[,sitename2], frequency=12)
    pet <- thornthwaite(ta, lat, na.rm=T)
    cwdiff <- ppt - pet
    for (j in 1:length(int_steps1)) {
        int_per <- int_steps1[j]
        spei_int <- spei(ts(cwdiff, frequency=12, start=c(1981, 1)), int_per, na.rm=TRUE)
        if (plot_spei){
            plot(spei_int)
        }
        spei_int <- xts(as.vector(spei_int$fitted),  index(df_tmean))
        colnames(spei_int) <- paste('SPEI_monthly_', as.character(int_per), sep='')
        if (j==1){
            spei_site <- spei_int
        } else {
            spei_site <- cbind(spei_site, spei_int)
        }
    }
    # There may be both infinite and NA values in the output
    # Convert -Inf to NA
    print(sum(is.na(spei_site)))
    spei_site[!is.finite(spei_site)] <- NA
    print(sum(is.na(spei_site)))
    # Actually there are no NA values - ignore this
    
    # Interpolate over NA values
    #spei_site_interp <- na.approx(spei_site)
    
    # Write files
    outfile <- paste('../processed_data/spei/SPEI_PRISM_monthly_', sitename1,
                     '.csv', sep='')
    write.zoo(spei_site, file = outfile,
              index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
    
    #outfile <- paste('../processed_data/spei/SPEI_PRISM_monthly_', sitename1,
    #                 '_nainterp.csv', sep='')
    #write.zoo(spei_site_interp, file = outfile,
    #          index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
}

[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
